# DES CUDA Application

## Usage

```
./des_cuda <encrypt|decrypt> <input_file> <output_file>
```

Where:
- `encrypt|decrypt`: Mode of operation ('encrypt' to encrypt the input file, 'decrypt' to decrypt it)
- `input_file`: Path to the file to be encrypted/decrypted
- `output_file`: Path where the encrypted/decrypted output will be saved

---

In [24]:
def compile_and_run_kernel(kernel_number, mode, input_file_name, output_file_name=None, analytics=False):
    import os
    
    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")
    # Create paths
    kernel_src = os.path.join(cwd, f"cuda_kernels/kernel_{kernel_number}.cu")
    kernel_exe = os.path.join(cwd, f"cuda_kernels/bin/kernel{kernel_number}.exe")
    
    # Ensure input and output file paths are correct
    input_file = os.path.join(cwd, f"Input_Files/{input_file_name}.txt")
    if output_file_name is None:
        if mode == "encrypt":
            output_file_name = f"{input_file_name}_encrypted"
        else:  # decrypt mode
            output_file_name = f"{input_file_name}_decrypted"
    output_file = os.path.join(cwd, f"Input_Files/{output_file_name}.txt")
    
    # Create bin directory if it doesn't exist
    os.makedirs(os.path.dirname(kernel_exe), exist_ok=True)
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Compile
    !nvcc "{kernel_src}" -o "{kernel_exe}"
    
    # Run with analytics if requested, otherwise run normally
    if analytics:
        # Create analytics_Bin directory if it doesn't exist
        analytics_dir = os.path.join(cwd, "analytics_Bin")
        os.makedirs(analytics_dir, exist_ok=True)
        
        # Set profile output path inside analytics_Bin folder
        profile_output = os.path.join(analytics_dir, f"profile_{input_file_name}_{mode}")
        
        # Run with nsys profiling
        !nsys profile --sample=none --trace=cuda --force-overwrite=true --stats=true --output="{profile_output}" "{kernel_exe}" "{mode}" "{input_file}" "{output_file}"
        print(f"Analytics data saved to {profile_output}")
    else:
        # Run normally
        !"{kernel_exe}" "{mode}" "{input_file}" "{output_file}"
    
    print(f"{mode.capitalize()}ed file saved to {output_file}")
    return output_file

# Example usage:
# output = compile_and_run_kernel("DES", "encrypt", "plaintext", analytics=True)

In [ ]:
import numpy as np
import os
import time

def compare_output_files(expected_file, actual_file, tolerance=1e-5, verbose=False):
    """
    Compare the output values in two text files to check if they're identical within a tolerance range.
    
    Parameters:
    - expected_file: Path to the file containing expected values
    - actual_file: Path to the file containing actual values from your implementation
    - tolerance: Maximum allowed difference between corresponding values
    - verbose: Whether to print details about the comparison
    
    Returns:
    - True if files match within tolerance, False otherwise
    """
    try:
        # Read expected output
        with open(expected_file, 'r') as f:
            expected_content = f.read().strip()
            expected_values = np.array([float(x) for x in expected_content.split()])
        
        # Read actual output
        with open(actual_file, 'r') as f:
            actual_content = f.read().strip()
            actual_values = np.array([float(x) for x in actual_content.split()])
        
        # Check if arrays have the same length
        if len(expected_values) != len(actual_values):
            print(f"❌ FAIL: Output files have different lengths! Expected: {len(expected_values)}, Actual: {len(actual_values)}")
            return False
        
        # Calculate absolute differences
        diff = np.abs(expected_values - actual_values)
        max_diff = np.max(diff)
        mean_diff = np.mean(diff)
        
        # Check if values match within tolerance
        match = np.allclose(expected_values, actual_values, rtol=0, atol=tolerance)
        
        if match:
            print(f"✅ PASS: Output values match within tolerance {tolerance}")
            if verbose:
                print(f"  - Maximum difference: {max_diff:.8e}")
                print(f"  - Average difference: {mean_diff:.8e}")
        else:
            print(f"❌ FAIL: Output values differ by more than tolerance {tolerance}")
            print(f"  - Maximum difference: {max_diff:.8e}")
            print(f"  - Average difference: {mean_diff:.8e}")
            
            if verbose:
                # Find and print the first few mismatched elements
                mismatched_indices = np.where(diff > tolerance)[0]
                print(f"  - Found {len(mismatched_indices)} mismatched values")
                for i in range(min(5, len(mismatched_indices))):
                    idx = mismatched_indices[i]
                    print(f"  - Index {idx}: Expected {expected_values[idx]:.8f}, Actual {actual_values[idx]:.8f}, Diff {diff[idx]:.8e}")
        
        return match
    
    except Exception as e:
        print(f"❌ FAIL: Error during comparison: {str(e)}")
        return False

# Example usage:
def verify_kernel_output(kernel_num, input_base, mask_base):
    """
    Verify the output of a kernel against the expected output.
    
    Parameters:
    - kernel_num: Kernel number (1, 2, or 3)
    - input_base: Base filename for input (without extension)
    - mask_base: Base filename for mask (without extension)
    """

    # Construct file paths
    base_path = "./Generator_TestCases/Convolution/"
    expected_file = f"{base_path}{input_base}_expected_output.txt"
    actual_file = f"./Output_TestCases/{input_base}_input_mask{mask_base}_k{kernel_num}_o.txt"

    print(f"Verifying kernel {kernel_num} output...")
    # Run comparison
    result = compare_output_files(expected_file, actual_file, tolerance=1e-1, verbose=True)
    return result

# Example: verify_kernel_output(1, "conv_v1000_m3_input", "conv_v1000_m3_mask")

### **DES Encryption and Decryption**

The following examples demonstrate the DES encryption and decryption functionality.

In [31]:
# Example 1: Encrypt a plaintext file
# compile_and_run_kernel("DES", "encrypt", "plaintext0", analytics=False)

# Example 2: Decrypt the encrypted file
compile_and_run_kernel("DES", "decrypt", "plaintext0_encrypted", analytics=False)

Current working directory: e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des
kernel_DES.cu
tmpxft_0000662c_00000000-10_kernel_DES.cudafe1.cpp
   Creating library e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\cuda_kernels\bin\kernelDES.lib and object e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\cuda_kernels\bin\kernelDES.exp
Decrypting 230 blocks with 256 threads in 1 cuda blocks...Decrypted file saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted_decrypted.txt


'e:\\02_Learn\\01_University\\Senior-1 Spring\\Current\\Parallel Computing\\Labs\\Des\\Input_Files/plaintext0_encrypted_decrypted.txt'


Decryption complete. Output written to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted_decrypted.txt


### **Performance Analysis**

Run with analytics to profile the encryption and decryption performance.

In [ ]:
# Profile encryption performance on a larger file
compile_and_run_kernel("DES", "encrypt", "large_plaintext", analytics=True)

# Profile decryption performance
compile_and_run_kernel("DES", "decrypt", "large_plaintext_encrypted", analytics=True)

### **Round-trip Test**

Test that encrypting and then decrypting a file recovers the original content.

In [ ]:
def run_round_trip_test(kernel_number, input_file_name):
    import os
    import filecmp
    
    print(f"Running round-trip test on {input_file_name}...")
    
    # First encrypt the input file
    encrypted_file = compile_and_run_kernel(kernel_number, "encrypt", input_file_name)
    
    # Then decrypt the encrypted file
    decrypted_file = compile_and_run_kernel(kernel_number, "decrypt", f"{input_file_name}_encrypted", f"{input_file_name}_round_trip")
    
    # Compare the original input with the decrypted output
    cwd = os.getcwd()
    original_file = os.path.join(cwd, f"Input_Files/{input_file_name}.txt")
    decrypted_output = os.path.join(cwd, f"Output_Files/{input_file_name}_round_trip.txt")
    
    if os.path.exists(original_file) and os.path.exists(decrypted_output):
        if filecmp.cmp(original_file, decrypted_output):
            print("✅ PASS: Round-trip test successful! Decrypted file matches the original.")
        else:
            print("❌ FAIL: Round-trip test failed. Decrypted file differs from the original.")
    else:
        print("❌ FAIL: One or both files don't exist.")

# Run the round-trip test
run_round_trip_test("DES", "plaintext0")